In [28]:
import pandas as pd
import statsmodels.api as sm
import yfinance as yf

# --- 1. Download stock data ---
ticker = "AAPL"
stock = yf.download(ticker, start="2020-01-01", end="2025-01-01")

# Handle flat vs MultiIndex columns, preferring Adj Close, else Close
if isinstance(stock.columns, pd.MultiIndex):
    if ("Adj Close", ticker) in stock.columns:
        stock = stock[("Adj Close", ticker)]
    elif ("Close", ticker) in stock.columns:
        stock = stock[("Close", ticker)]
    else:
        raise KeyError(f"No Adj Close or Close found in MultiIndex columns: {stock.columns}")
else:
    if "Adj Close" in stock.columns:
        stock = stock["Adj Close"]
    elif "Close" in stock.columns:
        stock = stock["Close"]
    else:
        raise KeyError(f"No Adj Close or Close found in flat columns: {stock.columns}")

returns = stock.pct_change().dropna()

# --- 2. Download Fama-French 5 factors ---
url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_CSV.zip"
ff = pd.read_csv(url, skiprows=3)

# --- Clean footer / non-numeric rows ---
# Keep only rows where 'Mkt-RF' is a number
ff = ff[pd.to_numeric(ff["Mkt-RF"], errors="coerce").notna()]

# Convert all to numeric
ff = ff.apply(pd.to_numeric, errors="coerce")

# Set index to datetime
ff.index = pd.to_datetime(ff["Unnamed: 0"].astype(str), format="%Y%m%d")
ff = ff.drop(columns=["Unnamed: 0"])

# --- 3. Align stock returns and factors ---
ff = ff.loc[returns.index]   # align by trading dates
y = (returns * 100 - ff["RF"])  # stock excess return (%)
X = ff.drop(columns=["RF"])
X = sm.add_constant(X)

# --- 4. Run regression ---
model = sm.OLS(y, X).fit()

# --- 5. Print results ---
print(model.summary())

# Nicely formatted table of betas + t-stats
results = pd.DataFrame({
    "coef": model.params,
    "tstat": model.tvalues,
    "pval": model.pvalues
})
print("\n=== Factor Loadings ===")
print(results.round(4))


[*********************100%***********************]  1 of 1 completed


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     636.3
Date:                Mon, 25 Aug 2025   Prob (F-statistic):               0.00
Time:                        21:49:14   Log-Likelihood:                -1856.6
No. Observations:                1257   AIC:                             3725.
Df Residuals:                    1251   BIC:                             3756.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0283      0.030      0.943      0.3

In [40]:
import pandas as pd
import statsmodels.api as sm
import yfinance as yf

# --- 1. Download stock data ---
ticker = "DECK"   # change this to any ticker
stock = yf.download(ticker, start="2020-01-01", end="2025-01-01")

# Handle flat vs MultiIndex columns
if isinstance(stock.columns, pd.MultiIndex):
    if ("Adj Close", ticker) in stock.columns:
        stock = stock[("Adj Close", ticker)]
    elif ("Close", ticker) in stock.columns:
        stock = stock[("Close", ticker)]
    else:
        raise KeyError(f"No Adj Close/Close in MultiIndex columns: {stock.columns}")
else:
    if "Adj Close" in stock.columns:
        stock = stock["Adj Close"]
    elif "Close" in stock.columns:
        stock = stock["Close"]
    else:
        raise KeyError(f"No Adj Close/Close in flat columns: {stock.columns}")

returns = stock.pct_change().dropna()

# --- 2. Download Fama-French 5 Factors ---
url_ff5 = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_CSV.zip"
ff5 = pd.read_csv(url_ff5, skiprows=3)
ff5 = ff5.rename(columns=lambda x: x.strip())  # strip whitespace
ff5 = ff5[pd.to_numeric(ff5["Mkt-RF"], errors="coerce").notna()]
ff5.index = pd.to_datetime(ff5["Unnamed: 0"].astype(str), format="%Y%m%d")
ff5 = ff5.drop(columns=["Unnamed: 0"])
ff5 = ff5.apply(pd.to_numeric, errors="coerce")

# --- 3. Download Momentum (UMD) ---
url_mom = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Momentum_Factor_daily_CSV.zip"
mom = pd.read_csv(url_mom, skiprows=13)
mom = mom.rename(columns=lambda x: x.strip())  # clean up
mom = mom[pd.to_numeric(mom["Mom"], errors="coerce").notna()]
mom.index = pd.to_datetime(mom["Unnamed: 0"].astype(str), format="%Y%m%d")
mom = mom.drop(columns=["Unnamed: 0"])
mom = mom.apply(pd.to_numeric, errors="coerce")

# --- 4. Merge factors ---
factors = ff5.join(mom, how="inner")
factors.rename(columns={"Mom": "MOM"}, inplace=True)

# --- 5. Align stock returns and factors ---
factors = factors.loc[returns.index]
y = (returns * 100 - factors["RF"])  # excess return (%)
factor_list = ["Mkt-RF", "SMB", "HML", "RMW", "CMA", "MOM"]

# --- 6. Multivariate regression ---
X_multi = sm.add_constant(factors[factor_list])
multi_model = sm.OLS(y, X_multi).fit()

# --- 7. Univariate regressions ---
uni_results = {}
for f in factor_list:
    X_uni = sm.add_constant(factors[[f]])
    model = sm.OLS(y, X_uni).fit()
    uni_results[f] = {
        "beta_uni": model.params[f],
        "tstat_uni": model.tvalues[f]
    }

# --- 8. Combine results ---
comparison = pd.DataFrame({
    "beta_uni": {f: uni_results[f]["beta_uni"] for f in factor_list},
    "tstat_uni": {f: uni_results[f]["tstat_uni"] for f in factor_list},
    "beta_multi": multi_model.params[factor_list],
    "tstat_multi": multi_model.tvalues[factor_list]
})

# --- 9. Print results ---
print(f"\n=== Factor Loadings Report for {ticker} (FF5 + MOM) ===")
print(comparison.round(4))
print("\nAlpha (multi):", round(multi_model.params['const'], 4))
print("R² (multi):", round(multi_model.rsquared, 4))


[*********************100%***********************]  1 of 1 completed



=== Factor Loadings Report for DECK (FF5 + MOM) ===
        beta_uni  tstat_uni  beta_multi  tstat_multi
Mkt-RF    1.2088    26.5489      1.0471      22.5757
SMB       0.9786    10.8752      0.8644       9.4787
HML      -0.1184    -1.7396      0.0019       0.0231
RMW      -0.5134    -4.3245      0.5275       4.6389
CMA      -1.4502   -11.4761     -1.0395      -7.3863
MOM      -0.2036    -3.5445      0.2153       4.2675

Alpha (multi): 0.1197
R² (multi): 0.4301
